![life](life.jpg)

You are a data analyst for a United Nations initiative focused on understanding global health trends. Your latest assignment is to explore and visualize life expectancy data from around the world, focusing on gender differences. 

Life expectancy can vary significantly over time and across different countries due to numerous factors, including advancements in medicine, a country's development level, and the impacts of conflicts. Interestingly, data consistently shows that women tend to live longer than men, raising intriguing questions. Could this be due to biological factors or perhaps because women generally care for their health better?

Your task is to explore these patterns and disparities. 

### The Data

The dataset contains information about life expectancy in various countries or areas, broken down by gender and time periods. The data is sourced from the _United Nations Population Division, Gender Statistics, Life Expectancy at Birth_.

#### UNdata.csv

| Column            | Meaning                                                                                        |
| ----------------- | ---------------------------------------------------------------------------------------------- |
| `Country.or.Area` | The name of the country or region being described.                                              |
| `Subgroup`        | The specific subgroup within the country or area (e.g., Female, Male).                          |
| `Year`            | The time period for the data provided (e.g., 2000-2005).                                        |
| `Source`          | The source of the data, specifying the UN publication or report where the data originated.      |
| `Unit`            | The unit of measurement for life expectancy.                  |
| `Value`           | The measured value for the life expectancy in the specified country, subgroup, and time period. |
| `Value.Footnotes` | Additional notes or comments related to the value, if any.                                      |


In [7]:
library(dplyr)
library(tidyr)
library(ggplot2)

life_expectancy = read.csv("datasets/UNdata.csv") # nolint
head(life_expectancy)

ERROR: Error in library(dplyr): there is no package called 'dplyr'


also installing the dependencies 'fansi', 'utf8', 'pkgconfig', 'withr', 'cli', 'generics', 'glue', 'lifecycle', 'magrittr', 'pillar', 'R6', 'rlang', 'tibble', 'tidyselect', 'vctrs'

Warning message:
"unable to access index for repository https://cran.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/3.6/PACKAGES'"Packages which are only available in source form, and may need
  compilation of C/C++/Fortran: 'fansi' 'utf8' 'cli' 'glue' 'magrittr'
  'rlang' 'tibble' 'tidyselect' 'vctrs' 'dplyr'


  These will not be installed


installing the source packages 'pkgconfig', 'withr', 'generics', 'lifecycle', 'pillar', 'R6'

Warning message in install.packages("dplyr"):
"installation of package 'lifecycle' had non-zero exit status"Warning message in install.packages("dplyr"):
"installation of package 'pillar' had non-zero exit status"

ERROR: Error in library(dplyr): there is no package called 'dplyr'


In [ ]:
# Start coding here
# Use as many cells as you need